In [ ]:
# Import necessary libraries
import os
import pickle
from langchain import ChatGroq, ChatMessageHistory, HumanMessage, AIMessage, ChatPromptTemplate, MessagesPlaceholder, RunnablePassthrough, RunnableBranch, StrOutputParser
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import create_stuff_documents_chain
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Setup the chat model
Model = 'llama3-8b-8192'
chat = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY, model_name=Model)

# Define the prompt templates
system_prompt = "You are a helpful assistant."
human_prompt = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system_prompt), ("human", human_prompt)])

# Build the conversational chain
chain = prompt | chat

# Load the document from the web
loader = WebBaseLoader("https://unctad.org/topic/competition-and-consumer-protection/un-guidelines-for-consumer-protection")
data = loader.load()

# Split the document text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

# Create the embedding model
embedding = OpenAIEmbeddings()

# Create the vector store and specify the persistence directory
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_dir)

# Load the vector store
vectorstore = Chroma(persist_directory=persist_dir, embedding_function=embedding)

# Define the retriever
retriever = vectorstore.as_retriever(k=4)

# Define the query transforming retriever chain
query_transform_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="messages"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.")
])

query_transforming_retriever_chain = RunnableBranch(
    (lambda x: len(x.get("messages", [])) == 1,
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    query_transform_prompt | chat | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

# Define the question answering prompt
question_answering_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="messages")
])

# Create the document chain
document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

# Combine the chains into the conversational retrieval chain
conversational_retrieval_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain,
).assign(
    answer=document_chain,
)

# Initialize the chat message history
demo_ephemeral_chat_history = ChatMessageHistory()

# Add initial user message
demo_ephemeral_chat_history.add_user_message("Summarise the Guidelines for Consumer Protection")

# Invoke the conversational retrieval chain
response = conversational_retrieval_chain.invoke({"messages": demo_ephemeral_chat_history.messages})

# Extract and print only the answer
print(response['answer'])

# Add the AI's answer to the chat history
demo_ephemeral_chat_history.add_ai_message(response["answer"])

# Continue the conversation
demo_ephemeral_chat_history.add_user_message("tell me more about that!")
response = conversational_retrieval_chain.invoke({"messages": demo_ephemeral_chat_history.messages})
print(response['answer'])

# Add user message to the chat history
demo_ephemeral_chat_history.add_user_message("thanks for that, so how is the consumer protection guaranteed")

# Invoke the conversational retrieval chain and extract only the answer
response = conversational_retrieval_chain.invoke({"messages": demo_ephemeral_chat_history.messages})
print(response['answer'])